In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import joblib
import os
import snowflake.connector
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
# import lightgbm and xgboost 
import lightgbm as lgb 
from lightgbm import LGBMRegressor
import xgboost as xgb 
from xgboost import plot_importance
from matplotlib import pyplot

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

#RMSE/MAE computation using sklearn library
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy
from xgboost import XGBRegressor
from IPython.display import display, HTML
from sklearn.model_selection import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from statistics import mean
from itertools import product

In [3]:
pd_query="""
select * from  scratch.riders.final_prediction_feat_set 
"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(pd_query)
df = cur.fetch_pandas_all()




Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZLLbtswEEV%2FRWDXEinZSWvCsuFXEKN2rdpOCnjHUGOFMUWqJBU5%2FfpSfgDpIll0QYAg78yZmTv94bGUwSsYK7RKURwRFIDiOheqSNHD9i78hgLrmMqZ1ApS9AYWDQd9y0pZ0VHtntUaftdgXeATKUvbjxTVRlHNrLBUsRIsdZxuRssFTSJCmbVgnMehS0huhWc9O1dRjJumiZpOpE2BE0IIJj3sVa3kC3qHqD5nVEY7zbW8hhx9Tx8gYky6LcIrPCG7BI6FOo%2FgM8rTWWTp%2FXabhdlqs0XB6NrdRCtbl2A2YF4Fh4f14lyA9RVMZ4v542y9WkVQhxyUM0yGcWSVbvaSHYDrsqqdTx35G95DjqUuhB%2FYfJqi6iByeRTl%2FrAXR73ojdUfe99xBcwM3Cx1d7d76SU%2Fl0%2Fr8fdfLOtyFDxe7U1ae%2BfW1jBXranOP5GkG5KvYRxv44R2bighUfc22aFg6k0VirlT5LVyxrmulbNRoXUh4VSfxq0jCRZ5NfRH5OmExC%2Fq0Lvl6Lwp9MQ0g%2F%2Fqv4%2Ffp7hs3g9vxnyaaSn4W3CnTcncx17FUXx6EXm4P0kplEzIUZ4bsNZ7JqVuJgaY8wvuTA0ID87Uf1d88Bc%3D&RelayState=59257 to authenticate..

In [4]:
print(df.shape)

df.sample(2)


(40341, 29)


,ZONE_CODE,CITY_NAME,HOUR_OF_DAY,WEEK_OF_DAY,LATE_BY_5_MINS_PERC_LAST_14_DYS,P50_RESTO_CUSTOMER_MINS_LAST_90_DYS,AVG_WAIT_AT_CUSTOMER_LAST_90_DYS,AVG_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS,AVG_TEMPERATURE_LAST_90_DYS,P10_RIDER_TO_RESTAURANT_MINS_LAST_30_DYS,...,P70_RESTO_CUSTOMER_MINS_LAST_14_DYS_SAME_HR,AVG_TEMPERATURE_LAST_14_DYS_SAME_HR,AVG_TEMPERATURE_LAST_5_DYS_SAME_HR,AVG_WAIT_AT_CUSTOMER_LAST_3_DYS_SAME_HR,P50_EOD_LAST_180_DYS_SAME_HR_WOD,P20_RIDER_TO_RESTAURANT_MINS_LAST_180_DYS_SAME_HR_WOD,P10_ERAT_LAST_90_DYS_SAME_HR_WOD,P70_RESTO_CUSTOMER_MINS_LAST_90_DYS_SAME_HR_WOD,P10_RIDER_TO_RESTAURANT_MINS_LAST_14_DYS_SAME_HR_WOD,P10_FPT_LAST_5_DYS_SAME_HR_WOD
25804,SDQ,Kuwait,9,Sun,0.245481,13.0,4.791124,5.882764,32.897729,0.0,...,16.199,40.806253,40.041902,6.184211,32.516667,2.2,520.0,19.0,3.0,2.0
9931,SRQA,Sharjah,10,Sun,0.263736,14.0,4.718676,10.685198,31.176671,5.0,...,10.800,39.960000,38.080000,4.016667,49.933333,14.0,640.0,17.0,NaN,NaN


In [5]:
xgb_model=joblib.load('/Users/richakumari/Desktop/Clustering/RET_prediction/RETfinal_v1_1.pkl')

In [6]:
cols=[    
'P50_AOD_LAST_14_DYS_SAME_HR',
'AVG_FPT_LAST_21_DYS',
'P70_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS_SAME_HR',
'AVG_ERAT_LAST_14_DYS_SAME_HR',
'AVG_ERAT_LAST_90_DYS_SAME_HR',
'P50_EOD_LAST_180_DYS_SAME_HR_WOD',
'AVG_WAIT_AT_CUSTOMER_LAST_3_DYS_SAME_HR',
'AVG_ERAT_LAST_5_DYS',
'P50_RESTO_CUSTOMER_MINS_LAST_90_DYS',
'P90_AOD_LAST_180_DYS_SAME_HR',
'AVG_WAIT_AT_CUSTOMER_LAST_90_DYS',
'AVG_ERAT_LAST_21_DYS_SAME_HR',
'P10_RIDER_TO_RESTAURANT_MINS_LAST_30_DYS',
'LATE_BY_5_MINS_PERC_LAST_14_DYS',
'P70_RESTO_CUSTOMER_MINS_LAST_14_DYS_SAME_HR',
'P10_RIDER_TO_RESTAURANT_MINS_LAST_14_DYS_SAME_HR_WOD',
'P10_ERAT_LAST_90_DYS_SAME_HR_WOD',
'LATE_BY_10_MINS_PERC_LAST_3_DYS',
'AVG_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS',
'P70_RESTO_CUSTOMER_MINS_LAST_90_DYS_SAME_HR_WOD',
'P20_RIDER_TO_RESTAURANT_MINS_LAST_180_DYS_SAME_HR_WOD',
'AVG_TEMPERATURE_LAST_90_DYS',
'AVG_TEMPERATURE_LAST_14_DYS_SAME_HR',
'AVG_TEMPERATURE_LAST_5_DYS_SAME_HR',
'P10_FPT_LAST_5_DYS_SAME_HR_WOD',
]


In [7]:
x = df[cols]
df['RET_pred'] =xgb_model.predict(x)


In [8]:
#  caping the abnormally high and low valus
p999=df.groupby('CITY_NAME')['RET_pred'].transform(lambda x: x.quantile(0.999))
df['RET_pred'] = df['RET_pred'].clip(upper=p999)

# Cap values lower than 0 in RET_AVG to 5
df.loc[df['RET_pred'] <=5, 'RET_pred'] = 5


In [9]:
#  obtaining the UR values here 

In [10]:
#  in case AOD is null replace it by city average AOD for same time
AOD_avg ="""with base as(
select 
distinct 
lower(city_name) as city_name
,case when zone_code in
('ABAR','AKW','AMR','AMZ','ANH','AQ','AQPP','AQS','ATW','AWH','DBB','DCH','DD11','DDO','DEIR','DFC','DIC','DMEY','DSO',
'DSW','DT','DWS','FCOW','HCC','HPDT','JMH','KAR','MRDF','NAS','RAK','UMS','WAA','ZBL') then  'Deira and Downtown Cluster'
when zone_code in (
'AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM') then 'Sportcity and Marina Cluster'
else city_name end as Cluster_Assigned
,case 
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (12, 13) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu','Fri') then 'Peak Lunch'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu') then 'Peak Dinner'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Fri','Sat','Sun') then 'Super Peak Dinner'
     else 'Other' end as shift
,round(COALESCE(SUM(aod_mins_sum ), 0) / NULLIF(COALESCE(SUM(aod_mins_cnt ), 0), 0),2) AS AOD_AVG
    
from
PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
where  date (start_of_period_local) between (current_date -22) and (current_date-2)
and CNT_ERAT >0 -- use this filter to get only active zone

group by 1,2,3
)
,map as(
 select
 distinct 
 city_name
 ,zone_code
 ,case when zone_code in
('ABAR','AKW','AMR','AMZ','ANH','AQ','AQPP','AQS','ATW','AWH','DBB','DCH','DD11','DDO','DEIR','DFC','DIC','DMEY','DSO',
'DSW','DT','DWS','FCOW','HCC','HPDT','JMH','KAR','MRDF','NAS','RAK','UMS','WAA','ZBL') then  'Deira and Downtown Cluster'
when zone_code in (
'AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM') then 'Sportcity and Marina Cluster'
else city_name end as Cluster_Assigned

 ,case 
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (12, 13) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu','Fri') then 'Peak Lunch'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu') then 'Peak Dinner'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Fri','Sat','Sun') then 'Super Peak Dinner'
     else 'Other' end as shift
FROm
PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
where  date (start_of_period_local) between (current_date -22) and (current_date-2)
and CNT_ERAT >0 
) 

select 
map.*, base. AOD_AVG
from 
map
left join
base 
on map.Cluster_Assigned=base.Cluster_Assigned
and map.shift=base.shift



"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(AOD_avg)
AOD_df = cur.fetch_pandas_all()





Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZJNb9swDIb%2FiqGdbcluljVCnCBNWixA13iJEwy5qbLiapYlT5Trtr9%2Bcj6A7tAedhAgSC%2F5kHw5nr7UKngWFqTRKYojggKhuSmkLlO0ze%2FCaxSAY7pgymiRolcBaDoZA6tVQ2ete9Jr8acV4AKfSAPtP1LUWk0NAwlUs1oAdZxuZj%2FuaRIRygCEdR6HziEFSM96cq6hGHddF3VXkbElTgghmIywV%2FWSL%2Bgdovmc0VjjDDfqEvLie%2FoAEWMy6BFe4QnZOfBG6tMIPqM8nkRAv%2Bd5FmarTY6C2aW7udHQ1sJuhH2WXGzX96cCwFewuL1f7m7Xq1Uk2pAL7SxTYRyBNt1BsUpwUzet86kjf8MHUWBlSukHtlykqKlksdrZ1WM5rN82rlnc6LeqZXml9vvh4BfMt7s2q7bDbbb%2BacmAo2B3sTfp7V0CtGKpe1OdfyLJICTfwjjO44RefaVkFF3HyR4FC2%2Bq1MwdIy%2BVM85Nqx1EpTGlEsf6DO4dSbAsmqk%2FskjnJP6tq9GQo9Om0CPTTv6r%2FzF%2Bn%2BK8eQ%2FejOUiM0ry1%2BDO2Jq5j72Ko%2Fj4IovwcJRSUTOpZkVhBYD3TCnTza1gzi%2B4s61AeHKi%2Frvik78%3D&RelayState=59273 t

In [11]:
print(AOD_df.shape)
AOD_df.sample(2)

(801, 5)


,CITY_NAME,ZONE_CODE,CLUSTER_ASSIGNED,SHIFT,AOD_AVG
741,Doha,QATAMH,Doha,Super Peak Dinner,38.87
103,Ajman,AJFW,Ajman,Peak Lunch,34.12


In [12]:
# #  read the csv with the equations for each city
# current_directory = os.getcwd()
# file_name = 'eqn.csv'
# file_path = os.path.join(current_directory, file_name)


UR_eqn = pd.read_csv('/Users/richakumari/Desktop/Clustering/eqn.csv')
UR_eqn.head(1)


,Unnamed: 0,CLUSTER_ASSIGNED,Equation,MAE,MAPE,R2,RMSE,Overprediction %,Underprediction %
0,0,Sportcity and Marina Cluster,UR_ZONE_CORRECTED = -0.7073 + 0.0379 * AOD_AVG,0.041531,0.06317,0.770216,0.052446,51.285521,48.714479


In [13]:
clusters_to_expand_deira = ['ABAR', 'AKW', 'AMR', 'AMZ', 'ANH', 'AQ', 'AQPP', 'AQS', 'ATW', 'AWH', 'DBB', 'DCH', 'DD11',
                      'DDO', 'DEIR', 'DFC', 'DIC', 'DMEY', 'DSO', 'DSW', 'DT', 'DWS', 'FCOW', 'HCC', 'HPDT', 'JMH',
                      'KAR', 'MRDF', 'NAS', 'RAK', 'UMS', 'WAA', 'ZBL']

clusters_to_expand_sportcity =['AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM']
# Function to expand clusters
def expand_clusters(row):
    clusters_assigned = row['CLUSTER_ASSIGNED'].split(', ')  # Split if multiple clusters are present
    if 'Deira and Downtown Cluster' in clusters_assigned :
        expanded_rows = []
        for cluster in clusters_to_expand_deira:
            new_row = {'CLUSTER_ASSIGNED': cluster, 'Equation': row['Equation']}
            expanded_rows.append(new_row)
        return pd.DataFrame(expanded_rows)
    if 'Sportcity and Marina Cluster' in clusters_assigned :
        expanded_rows = []
        for cluster in clusters_to_expand_sportcity:
            new_row = {'CLUSTER_ASSIGNED': cluster, 'Equation': row['Equation']}
            expanded_rows.append(new_row)
        return pd.DataFrame(expanded_rows)
    else:
        return pd.DataFrame([row])

# Apply the function to each row and concatenate results
expanded_df = pd.concat([expand_clusters(row) for _, row in UR_eqn.iterrows()], ignore_index=True)


UR_eqn_df=expanded_df[['Equation','CLUSTER_ASSIGNED']]



def extract_components(equation):
    try:
        # Split by '=' to separate left-hand side and right-hand side
        lhs, rhs = equation.split('=')
        
        # Strip whitespace from both sides
        lhs = lhs.strip()
        rhs = rhs.strip()
        
        # Check if rhs contains '+'
        if '+' in rhs:
            # Split by '+' to separate constant and term with variable
            constant, term_with_var = [part.strip() for part in rhs.split('+')]
            
            # Extract variable and coefficient
            coefficient, variable = [part.strip() for part in term_with_var.split('*')]
            
            # Strip any additional whitespace from variable
            variable = variable.strip()
            
            # Convert constants to float
            constant = float(constant)
            coefficient = float(coefficient)
            
            # Return a Series with components
            return pd.Series([lhs, constant, coefficient, variable], index=['LHS', 'Constant', 'Coefficient', 'Variable'])
        else:
            raise ValueError("Unexpected format in equation: {}".format(equation))
    
    except Exception as e:
        print(f"Error processing equation '{equation}': {str(e)}")
        return pd.Series([None, None, None, None], index=['LHS', 'Constant', 'Coefficient', 'Variable'])

# Apply the function to each row and expand into new columns
UR_eqn_df[['LHS', 'Constant', 'Coefficient', 'Variable']] = UR_eqn_df['Equation'].apply(extract_components)
UR_eqn_df. head(2)


,Equation,CLUSTER_ASSIGNED,LHS,Constant,Coefficient,Variable
0,UR_ZONE_CORRECTED = -0.7073 + 0.0379 * AOD_AVG,AR,UR_ZONE_CORRECTED,-0.7073,0.0379,AOD_AVG
1,UR_ZONE_CORRECTED = -0.7073 + 0.0379 * AOD_AVG,DBRS,UR_ZONE_CORRECTED,-0.7073,0.0379,AOD_AVG


In [14]:
# Conditionally merge based on city
# Option 1: When city is 'Dubai', merge on cluster_id and cluster_assigned
condition_dubai = df['CITY_NAME'] == 'Dubai'
merged_dubai = pd.merge(df[condition_dubai], UR_eqn_df, left_on='ZONE_CODE', right_on='CLUSTER_ASSIGNED')

# Option 2: When city is not 'Dubai', merge on city and cluster_assigned
condition_not_dubai = df['CITY_NAME'] != 'Dubai'
merged_not_dubai = pd.merge(df[condition_not_dubai], UR_eqn_df, left_on='CITY_NAME', right_on='CLUSTER_ASSIGNED')

# Concatenate the results from both merges
final_merged_df = pd.concat([merged_dubai, merged_not_dubai])
final_merged_df.sample(2)

,ZONE_CODE,CITY_NAME,HOUR_OF_DAY,WEEK_OF_DAY,LATE_BY_5_MINS_PERC_LAST_14_DYS,P50_RESTO_CUSTOMER_MINS_LAST_90_DYS,AVG_WAIT_AT_CUSTOMER_LAST_90_DYS,AVG_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS,AVG_TEMPERATURE_LAST_90_DYS,P10_RIDER_TO_RESTAURANT_MINS_LAST_30_DYS,...,P70_RESTO_CUSTOMER_MINS_LAST_90_DYS_SAME_HR_WOD,P10_RIDER_TO_RESTAURANT_MINS_LAST_14_DYS_SAME_HR_WOD,P10_FPT_LAST_5_DYS_SAME_HR_WOD,RET_pred,Equation,CLUSTER_ASSIGNED,LHS,Constant,Coefficient,Variable
10499,ADMS,Abu Dhabi,16,Wed,0.162554,13.0,4.750090,5.738833,32.451298,0.0,...,15.000,1.0,5.0,23.691923,UR_ZONE_CORRECTED = -0.8693 + 0.0407 * AOD_AVG,Abu Dhabi,UR_ZONE_CORRECTED,-0.8693,0.0407,AOD_AVG
9485,NAS,Dubai,11,Sat,0.148331,15.0,4.335106,6.721891,32.539755,2.0,...,15.299,1.2,4.2,23.820694,UR_ZONE_CORRECTED = -0.6930 + 0.0354 * AOD_AVG,NAS,UR_ZONE_CORRECTED,-0.6930,0.0354,AOD_AVG


In [15]:
# Define a function to calculate Peak_Type
def calculate_peak_type(row):
    hour = row['HOUR_OF_DAY']
    weekday = row['WEEK_OF_DAY']
    
    if hour in [12, 13] and weekday in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']:
        return 'Peak Lunch'
    elif hour in [19, 20] and weekday in ['Mon', 'Tue', 'Wed', 'Thu']:
        return 'Peak Dinner'
    elif hour in [19, 20] and weekday in ['Fri', 'Sat', 'Sun']:
        return 'Super Peak Dinner'
    else:
        return 'Other'

# Apply the function to create the new column 'SHIFT'
final_merged_df['SHIFT'] = final_merged_df.apply(calculate_peak_type, axis=1)


#  based on zone X shift populate the AOD values 
final_merged_df = pd.merge(final_merged_df, AOD_df[['ZONE_CODE', 'SHIFT', 'AOD_AVG']], on=['ZONE_CODE', 'SHIFT'], how='left')


In [16]:
final_merged_df['UR']=final_merged_df['Constant']+final_merged_df['Coefficient'].astype(float)*final_merged_df['AOD_AVG'].astype(float)

# Calculate the mean UR for each CITY_NAME and SHIFT group
mean_ur_by_city_shift = final_merged_df.groupby(['CITY_NAME', 'SHIFT'])['UR'].transform('mean')

# Fill NaN values in UR with the calculated mean values
final_merged_df['UR_updated'] = final_merged_df['UR'].fillna(mean_ur_by_city_shift)


#  case when UR is nan  replace with avg of the shift X city

final_merged_df['pred_TP']=final_merged_df['UR_updated']*(60/final_merged_df['RET_pred'])

In [17]:
#  rename the weekdays from mon-sun as 1-7 for Dubai, and sun-mon as 1-7 for Qatar and Kuwait
#  and adjust the hour of day  from 0-6 as 24-30 of next days

In [18]:

# Define the mapping for WEEK_OF_DAY for normal cities
week_day_mapping = {'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6, 'Sun': 7}
reverse_week_day_mapping = {v: k for k, v in week_day_mapping.items()}

# Define the mapping for WEEK_OF_DAY for Kuwait and Qatar
week_day_mapping_kw_qt = {'Sun': 1, 'Mon': 2, 'Tue': 3, 'Wed': 4, 'Thu': 5, 'Fri': 6, 'Sat': 7}
reverse_week_day_mapping_kw_qt = {v: k for k, v in week_day_mapping_kw_qt.items()}

# Adjust HOUR_OF_DAY and WEEK_OF_DAY based on the conditions
def adjust_hour_and_day(row):
    if row['CITY_NAME'] in ['Kuwait', 'Doha']:
        if 0 <= row['HOUR_OF_DAY'] <= 6:
            adjusted_hour = row['HOUR_OF_DAY'] + 24
            adjusted_day_numeric = week_day_mapping_kw_qt[row['WEEK_OF_DAY']] - 1
            if adjusted_day_numeric == 0:
                adjusted_day_numeric = 7
            adjusted_day = reverse_week_day_mapping_kw_qt[adjusted_day_numeric]
        else:
            adjusted_hour = row['HOUR_OF_DAY']
            adjusted_day = row['WEEK_OF_DAY']
        adjusted_day_numeric = week_day_mapping_kw_qt[adjusted_day]
    else:
        if 0 <= row['HOUR_OF_DAY'] <= 6:
            adjusted_hour = row['HOUR_OF_DAY'] + 24
            adjusted_day_numeric = week_day_mapping[row['WEEK_OF_DAY']] - 1
            if adjusted_day_numeric == 0:
                adjusted_day_numeric = 7
            adjusted_day = reverse_week_day_mapping[adjusted_day_numeric]
        else:
            adjusted_hour = row['HOUR_OF_DAY']
            adjusted_day = row['WEEK_OF_DAY']
        adjusted_day_numeric = week_day_mapping[adjusted_day]
    
    return pd.Series([adjusted_hour, adjusted_day, adjusted_day_numeric])

final_merged_df[['ADJUSTED_HOUR_OF_DAY', 'ADJUSTED_WEEK_OF_DAY', 'ADJUSTED_WEEK_OF_DAY_NUMERIC']] = final_merged_df.apply(adjust_hour_and_day, axis=1)



In [22]:

final_merged_df.groupby([ 'CITY_NAME'])['pred_TP'].mean().reset_index()

,CITY_NAME,pred_TP
0,Abu Dhabi,1.344708
1,Ajman,0.888817
2,Al Ain,0.966097
3,Doha,1.074169
4,Dubai,1.365396
5,Kuwait,1.499036
6,Ras Al Khaimah,1.144219
7,Sharjah,1.217834


In [20]:
downlaod=final_merged_df[['CITY_NAME','ZONE_CODE','ADJUSTED_WEEK_OF_DAY_NUMERIC','ADJUSTED_HOUR_OF_DAY','pred_TP']]
downlaod.to_csv('TP_all_cities.csv')

In [21]:
x= final_merged_df[(final_merged_df['pred_TP'].isna() | (final_merged_df['pred_TP'] == ''))]
x




,ZONE_CODE,CITY_NAME,HOUR_OF_DAY,WEEK_OF_DAY,LATE_BY_5_MINS_PERC_LAST_14_DYS,P50_RESTO_CUSTOMER_MINS_LAST_90_DYS,AVG_WAIT_AT_CUSTOMER_LAST_90_DYS,AVG_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS,AVG_TEMPERATURE_LAST_90_DYS,P10_RIDER_TO_RESTAURANT_MINS_LAST_30_DYS,...,Coefficient,Variable,SHIFT,AOD_AVG,UR,UR_updated,pred_TP,ADJUSTED_HOUR_OF_DAY,ADJUSTED_WEEK_OF_DAY,ADJUSTED_WEEK_OF_DAY_NUMERIC
